### Imports

In [ ]:
# ML imports
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report, precision_score, recall_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Data analysis and stats imports
import numpy as np
import pandas as pd
from scipy.stats import expon, reciprocal
from scipy.spatial.distance import pdist, squareform

# Data visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

from common_language import _LANGUAGES
import processing as prlib

### Get data

In [ ]:
df = prlib.get_preprocessed_data()
df = df.reset_index().iloc[:, 2:]
df.rename(columns={'0': 'label'}, inplace=True)

df_without_label = df.iloc[:, 0:-1]

Relabeling

In [ ]:
# relabel by types of language
print(df['label'].unique())
group_labels = {
    'Arabic': 'Languages of the Caucasus and Middle East', 'Basque': 'Other European Languages',
    'Breton': 'Constructed and Isolate Languages', 'Catalan': 'Other European Languages',
    'Chinese_China': 'East Asian Languages', 'Chinese_Hongkong': 'East Asian Languages',
    'Chinese_Taiwan': 'East Asian Languages', 'Chuvash': 'Turkic Languages',
    'Czech': 'Slavic Languages', 'Dhivehi': 'Diverse Asian and Pacific Languages',
    'Dutch': 'Germanic Languages', 'English': 'Germanic Languages',
    'Esperanto': 'Constructed and Isolate Languages', 'Estonian': 'Other European Languages',
    'French': 'Romance European Languages', 'Frisian': 'African and Other Languages',
    'Georgian': 'Languages of the Caucasus and Middle East', 'German': 'Germanic Languages',
    'Greek': 'Romance European Languages', 'Hakha_Chin': 'Diverse Asian and Pacific Languages',
    'Indonesian': 'Diverse Asian and Pacific Languages', 'Interlingua': 'Constructed and Isolate Languages',
    'Italian': 'Romance European Languages', 'Japanese': 'East Asian Languages',
    'Kabyle': 'African and Other Languages', 'Kinyarwanda': 'African and Other Languages',
    'Kyrgyz': 'Turkic Languages', 'Latvian': 'Other European Languages',
    'Maltese': 'Languages of the Caucasus and Middle East', 'Mongolian': 'Diverse Asian and Pacific Languages',
    'Persian': 'Languages of the Caucasus and Middle East', 'Polish': 'Slavic Languages',
    'Portuguese': 'Romance European Languages', 'Romanian': 'Romance European Languages',
    'Romansh_Sursilvan': 'Constructed and Isolate Languages', 'Russian': 'Slavic Languages',
    'Sakha': 'Turkic Languages', 'Slovenian': 'Slavic Languages',
    'Spanish': 'Romance European Languages', 'Swedish': 'Germanic Languages',
    'Tamil': 'African and Other Languages', 'Tatar': 'Turkic Languages',
    'Turkish': 'Turkic Languages', 'Ukranian': 'Slavic Languages', 'Welsh': 'Other European Languages'
}

# Apply the mapping to the DataFrame
df['label'] = df['label'].map(group_labels)
#print(df['label'].unique())
df.groupby('label').count()

Check multicolinearity

In [ ]:
corr = df_without_label.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

PCA or MDS

In [ ]:
df['label']

In [ ]:
# PCA
df_pca = prlib.get_PCs(df_without_label, 99)
df_pca['label'] = df['label']

# MDS with mahalanobis
#df_mds = mds_mahalanobis(df_without_label, 70)
#df_mds['label'] = df['label']

In [ ]:
df_pca

## Models

### Training

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_pca.iloc[:, 0:-1], df_pca['label'], stratify=df_pca['label'], test_size=0.3, random_state=42)

# Model initialization
svm = SVC(verbose=3, kernel='linear', random_state=42)
svm_poly = SVC(verbose=3, kernel='poly', random_state=42)
svm_rbf = SVC(verbose=3, kernel='rbf', random_state=42)
random_forest = RandomForestClassifier(verbose=3, random_state=42)

# Models fiting
print("Training linear SVM")
svm.fit(X_train, Y_train)
print()
print('Training poly SVM')
svm_poly.fit(X_train, Y_train)
print()
print('Training rbf SVM')
svm_rbf.fit(X_train, Y_train)
print()
print("Training RFC")
random_forest.fit(X_train, Y_train)


### Predictions

In [ ]:
# Models prediction
svm_predictions = svm.predict(X_test)
poly_svm_pred = svm_poly.predict(X_test)
rbf_svm_pred = svm_rbf.predict(X_test)
random_forest_predictions = random_forest.predict(X_test)

### Metrics

In [ ]:
svm_accuracy = accuracy_score(Y_test, svm_predictions)
poly_svm_acc = accuracy_score(Y_test, poly_svm_pred)
rbf_svm_acc = accuracy_score(Y_test, rbf_svm_pred)
random_forest_accuracy = accuracy_score(Y_test, random_forest_predictions)

svm_f1 = f1_score(Y_test, svm_predictions, average='macro')
poly_svm_f1 = f1_score(Y_test, poly_svm_pred, average='macro')
rbf_svm_f1 = f1_score(Y_test, rbf_svm_pred, average='macro')
rdf_f1 = f1_score(Y_test, random_forest_predictions, average='macro')

svm_precision_score = precision_score(Y_test, svm_predictions, average='macro')
poly_svm_precision_score = precision_score(Y_test, poly_svm_pred, average='macro')
rbf_svm_precision_score = precision_score(Y_test, rbf_svm_pred, average='macro')
random_forest_precision_score = precision_score(Y_test, random_forest_predictions, average='macro')

svm_recall = recall_score(Y_test, svm_predictions, average='macro')
poly_svm_recall = recall_score(Y_test, poly_svm_pred, average='macro')
rbf_svm_recall = recall_score(Y_test, rbf_svm_pred, average='macro')
rdf_recall = recall_score(Y_test, random_forest_predictions, average='macro')

print(f'linear SVM accuracy: {svm_accuracy}')
print(f'poly SVM accuracy: {poly_svm_acc}')
print(f'rbf SVM accuracy: {rbf_svm_acc}')
print(f'RDF accuracy: {random_forest_accuracy}')
print()
print(f'linear SVM f1: {svm_f1}')
print(f'poly SVM f1: {poly_svm_f1}')
print(f'rbf SVM f1: {rbf_svm_f1}')
print(f'RDF f1: {rdf_f1}')
print()
print(f'linear SVM precision: {svm_precision_score}')
print(f'poly SVM precision: {poly_svm_precision_score}')
print(f'rbf SVM precision: {rbf_svm_precision_score}')
print(f'RDF precision: {random_forest_precision_score}')
print()
print(f'linear SVM recall: {svm_recall}')
print(f'poly SVM recall: {poly_svm_recall}')
print(f'rbf SVM recall: {rbf_svm_recall}')
print(f'RDF recall: {rdf_recall}')

### Random Search

RandomizedSearch SVM

In [ ]:
param_grid_svm = {
    'C': reciprocal(0.001, 1000),
    'gamma': expon(scale=1.0),
    'kernel': ['linear', 'rbf', 'poly']
}

svm_clf = SVC(random_state=42)
random_search_svm = RandomizedSearchCV(svm_clf, param_distributions=param_grid_svm, n_iter=20, verbose=3, cv=5, random_state=42, n_jobs=-1, scoring = 'f1_macro')
random_search_svm.fit(X_train, Y_train)
print("Best parameters for SVM:", random_search_svm.best_params_)
print("Best score:", random_search_svm.best_score_)

RandomizedSearch Random Forest Classifier

In [ ]:
param_grid_rf = {
    'n_estimators': [80, 100, 200],
    'max_depth': [3, 4, 5, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

rfc = RandomForestClassifier(random_state=42)

random_search_rf = RandomizedSearchCV(rfc, param_distributions=param_grid_rf, n_iter=20, cv=5, verbose=3, random_state=42, n_jobs=-1, scoring = 'f1_macro')
random_search_rf.fit(X_train, Y_train)
print("Best parameters:", random_search_rf.best_params_)
print("Best score:", random_search_rf.best_score_)